In [1]:
import pandas as pd

# Load the provided CSV file
file_path = 'all_coins_data.csv'
all_coins_data = pd.read_csv(file_path)

# Step 1: Handle Missing Values
# Fill NaN values for 'fully_diluted_valuation', 'max_supply', and 'roi' with 0, since they represent missing data
all_coins_data['fully_diluted_valuation'].fillna(0, inplace=True)
all_coins_data['max_supply'].fillna(0, inplace=True)
all_coins_data['roi'].fillna(0, inplace=True)

# Step 2: Convert Date Columns to Datetime
all_coins_data['ath_date'] = pd.to_datetime(all_coins_data['ath_date'], errors='coerce')
all_coins_data['atl_date'] = pd.to_datetime(all_coins_data['atl_date'], errors='coerce')
all_coins_data['last_updated'] = pd.to_datetime(all_coins_data['last_updated'], errors='coerce')

# Step 3: Drop Unnecessary Columns
# Drop 'image' column as it is not useful for analysis
all_coins_data.drop(columns=['image'], inplace=True)

# Step 4: Rename Columns for Readability
all_coins_data.rename(columns={
    'ath': 'All_Time_High',
    'atl': 'All_Time_Low',
    'ath_change_percentage': 'ATH_Change_Percentage',
    'atl_change_percentage': 'ATL_Change_Percentage',
    'ath_date': 'ATH_Date',
    'atl_date': 'ATL_Date',
    'last_updated': 'Last_Updated',
}, inplace=True)

# Step 5: Fix Data Types if necessary
# Ensure numeric columns are in float format
numeric_columns = [
    'current_price', 'market_cap', 'fully_diluted_valuation', 'total_volume',
    'high_24h', 'low_24h', 'All_Time_High', 'All_Time_Low',
    'ATH_Change_Percentage', 'ATL_Change_Percentage'
]
all_coins_data[numeric_columns] = all_coins_data[numeric_columns].astype(float)

# Step 6: Handle Outliers
# Identify and cap extreme outliers for ATH and ATL change percentages
# Capping to a reasonable percentile (e.g., 99.5%) to reduce skew due to extreme outliers
for col in ['ATH_Change_Percentage', 'ATL_Change_Percentage']:
    upper_limit = all_coins_data[col].quantile(0.995)
    all_coins_data[col] = all_coins_data[col].clip(upper=-upper_limit)

# Display the cleaned data
print(all_coins_data.head())

# Optionally, save the cleaned dataframe to a new CSV file
# all_coins_data.to_csv('cleaned_all_coins_data.csv', index=False)


         id symbol      name  current_price    market_cap  market_cap_rank  \
0   bitcoin    btc   Bitcoin     101205.000  2.000380e+12                1   
1  ethereum    eth  Ethereum       3908.910  4.699840e+11                2   
2    ripple    xrp       XRP          2.410  1.373320e+11                3   
3    tether   usdt    Tether          1.001  1.358230e+11                4   
4    solana    sol    Solana        239.890  1.138580e+11                5   

   fully_diluted_valuation  total_volume    high_24h       low_24h  ...  \
0             2.122630e+12  1.434480e+11  103679.000  94870.000000  ...   
1             4.699840e+11  6.255699e+10    3946.580   3774.780000  ...   
2             2.406840e+11  2.759453e+10       2.470      2.210000  ...   
3             1.358230e+11  1.809590e+11       1.012      0.994958  ...   
4             1.412160e+11  1.160859e+10     244.110    224.400000  ...   

   total_supply    max_supply  All_Time_High  ATH_Change_Percentage  \
0  2.1000

In [2]:
import numpy as np

# Load the provided CSV file
all_coins_data = pd.read_csv('all_coins_data.csv')

# Extract relevant columns from all_coins_data
all_coins_data_subset = all_coins_data[['id', 'symbol', 'current_price', 'total_volume', 'high_24h', 'low_24h', 'last_updated']]

# Convert 'last_updated' to timestamp and add it to the new dataframe
all_coins_data_subset = all_coins_data_subset.copy()  # Create a copy to avoid potential pitfalls
all_coins_data_subset['timestamp'] = pd.to_datetime(all_coins_data_subset['last_updated'], errors='coerce').astype(np.int64) // 10**9

all_coins_transformed = all_coins_data_subset.rename(columns={
    'id': 'Asset_ID',
    'current_price': 'Close',
    'high_24h': 'High',
    'low_24h': 'Low',
    'total_volume': 'Volume'
})

# Create a numerical Asset_ID for the new data
# Assign new numerical IDs to the Asset_ID field, starting from 1
asset_id_mapping = {name: idx for idx, name in enumerate(all_coins_transformed['Asset_ID'].unique(), start=1)}
all_coins_transformed['Asset_ID'] = all_coins_transformed['Asset_ID'].map(asset_id_mapping)

all_coins_transformed['Asset_ID'] = all_coins_transformed['Asset_ID'].astype(float)

all_coins_transformed['Count'] = np.random.randint(500, 2000, size=len(all_coins_transformed))
all_coins_transformed['Open'] = all_coins_transformed['Close'] * np.random.uniform(0.98, 1.02, size=len(all_coins_transformed))

all_coins_transformed['Open'] = all_coins_transformed['Open'].fillna(0)  # Or any placeholder value

all_coins_transformed['VWAP'] = (all_coins_transformed['High'] + all_coins_transformed['Low'] + all_coins_transformed['Close']) / 3
all_coins_transformed['VWAP'] = all_coins_transformed['VWAP'].fillna(0)  # Handle NaN in VWAP

all_coins_transformed['group_num'] = 0  
all_coins_transformed['row_id'] = all_coins_transformed.index  # Row index as ID

# Reorder columns to match the desired structure
all_coins_transformed = all_coins_transformed[['timestamp', 'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'group_num', 'row_id']]

# Save the transformed data to a new CSV file
all_coins_transformed.to_csv('transformed_all_coins_data.csv', index=False)

# Optionally, print the first few rows to verify
print(all_coins_transformed.head())


    timestamp  Asset_ID  Count           Open        High           Low  \
0  1733418474       1.0   1474  100547.977924  103679.000  94870.000000   
1  1733418476       2.0    727    3844.632256    3946.580   3774.780000   
2  1733418474       3.0   1088       2.391609       2.470      2.210000   
3  1733418476       4.0   1695       0.989428       1.012      0.994958   
4  1733418473       5.0    956     237.955155     244.110    224.400000   

        Close        Volume          VWAP  group_num  row_id  
0  101205.000  1.434480e+11  99918.000000          0       0  
1    3908.910  6.255699e+10   3876.756667          0       1  
2       2.410  2.759453e+10      2.363333          0       2  
3       1.001  1.809590e+11      1.002653          0       3  
4     239.890  1.160859e+10    236.133333          0       4  
